<a href="https://colab.research.google.com/github/Harbiodun0122/OCR-for-Nigerian-Licence-Plate-/blob/master/LicensePlateOCR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Mount google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
# Install ultralytics and fast-plate-ocr
!pip install ultralytics fast-plate-ocr[onnx]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 33.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 3.8 MB/s eta 0:00:00


In [3]:
# Import libraries
import cv2, os, re, csv
from ultralytics import YOLO
from google.colab.patches import cv2_imshow
from fast_plate_ocr import LicensePlateRecognizer

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Fast plate OCR

In [4]:
# Load the fast plate ocr model
plate_recognizer = LicensePlateRecognizer('cct-xs-v1-global-model')

In [5]:
# Nigerian plate patterns
NIGERIAN_PLATE_PATTERNS = [
    r'^[A-Z]{3}\d{3}[A-Z]{2}$',      # AAA123AA
    r'^[A-Z]{2}\d{3}[A-Z]{3}$',      # AA123AAA
    r'^[A-Z]{2}\d{2}[A-Z]\d{2}$',    # AA12A34
    r'^\d{2}[A-Z]\d{2}[A-Z]{2}$',    # 12A34AA
    r'^[A-Z]{4}\d{3}$',              # AAAA123
]

def format_plate_pattern(text):
  """
  Fornat the plate number to the correct pattern.
  """

  cleaned = text.upper().replace(' ', '').replace('-', '')

  # AAA123AA -> AAA-123AA
  if re.match(NIGERIAN_PLATE_PATTERNS[0], cleaned):
    return f"{cleaned[:3]}-{cleaned[3:]}"

  # AA123AAA -> AA-123AAA
  elif re.match(NIGERIAN_PLATE_PATTERNS[1], cleaned):
    return f"{cleaned[:2]}-{cleaned[2:]}"

  # AA12A34 -> AA12-A34
  elif re.match(NIGERIAN_PLATE_PATTERNS[2], cleaned):
    return f"{cleaned[:4]}-{cleaned[4:]}"

  # 12A34AA -> 12A-34AA
  elif re.match(NIGERIAN_PLATE_PATTERNS[3], cleaned):
    return f"{cleaned[:3]}-{cleaned[3:]}"

  # AAAA123 -> AAAA-123
  elif re.match(NIGERIAN_PLATE_PATTERNS[4], cleaned):
    return f"{cleaned[:4]}-{cleaned[4:]}"

  # If pattern do not match, return text as it is
  else:
    return cleaned

### Run Inference on a video

In [13]:
# Setting the paths
BASE_DIR = "/content/drive/MyDrive/Licence Plate OCR/"
video_path = os.path.join(BASE_DIR, "traffic.mp4")
csv_path = os.path.join(BASE_DIR, "detected_plates.csv")
model_path = os.path.join(BASE_DIR, "licence_detection_output/train/weights/best.pt")

**Record plate numbers in a CSV file**

In [14]:
# Load already-saved plates if file exists
saved_plates = set()

if os.path.exists(csv_path):
    with open(csv_path, newline="", mode="r") as f:
        reader = csv.reader(f)
        next(reader, None)  # skip header
        for row in reader:
            if row:
                saved_plates.add(row[0])

# Open CSV in append mode
csv_file = open(csv_path, mode="a", newline="")
csv_writer = csv.writer(csv_file)

# Write header only once
if csv_file.tell() == 0:
    csv_writer.writerow(["plate_number"])

In [12]:
model = YOLO(model_path)

# Open the video file and get video details
videoCap = cv2.VideoCapture(video_path)
width = int(videoCap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(videoCap.get(cv2.CAP_PROP_FRAME_HEIGHT))
frame_per_second = int(videoCap.get(cv2.CAP_PROP_FPS))
num_frames = int(videoCap.get(cv2.CAP_PROP_FRAME_COUNT))

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(os.path.join(BASE_DIR, "License Plate OCR.mp4"), fourcc, frame_per_second, (width, height))

# Loop through the video frames
while videoCap.isOpened():
  # Read a frame from the video
  success, frame = videoCap.read()

  if success:
    # Run YOLO11 detection on the frame
    result = model(frame, task="detect")

    for box in result[0].boxes:
      # check confidence score is greater than 35%
      if box.conf[0] > 0.35:
        # get the coordinates
        [x1, y1, x2, y2] = box.xyxy[0]

        # convert to ints
        x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)

        cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)

        #crop detected plate
        plate_img = frame[y1:y2, x1:x2]
        plate = plate_recognizer.run(plate_img)

        # remove underscore used for padding
        plate = plate[0].replace("_", "")

        # properly format plate number
        formatted_plate = format_plate_pattern(plate)

        # Save to CSV only if new
        if formatted_plate and formatted_plate not in saved_plates:
          csv_writer.writerow([formatted_plate])
          saved_plates.add(formatted_plate)

        # Get text size for background
        text_size, _ = cv2.getTextSize(formatted_plate, cv2.FONT_HERSHEY_SIMPLEX, 1, 2)
        text_w, text_h = text_size

        # Draw blue background rectangle
        cv2.rectangle(frame, (x1, y1 - text_h - 10), (x1 + text_w + 10, y1), (255, 0, 0), -1)

        # Draw white text on top
        cv2.putText(frame, formatted_plate, (x1 + 5, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    out.write(frame)

  else:
    # Break the loop if the end of the video is reached
    break

# Release the video capture object and close the display window
videoCap.release()
out.release()
cv2.destroyAllWindows()
csv_file.close()


0: 384x640 (no detections), 395.4ms
Speed: 5.8ms preprocess, 395.4ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 388.2ms
Speed: 4.9ms preprocess, 388.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 387.0ms
Speed: 4.5ms preprocess, 387.0ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 393.2ms
Speed: 4.7ms preprocess, 393.2ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 400.9ms
Speed: 5.2ms preprocess, 400.9ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 392.7ms
Speed: 7.6ms preprocess, 392.7ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license_plate, 367.6ms
Speed: 6.3ms preprocess, 367.6ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 369.0ms
Speed: 4.6ms prepr